In [173]:
import os, sys
sys.path.append(os.path.join(os.path.expanduser('~'), 'Code'))
# sys.path

from lonelyboy.geospatial import plots as gsplt
from lonelyboy.geospatial import preprocessing as gspp
from lonelyboy.timeseries import lbtimeseries as tspp
from lonelyboy.geospatial import group_patterns as gsgp
from lonelyboy.geospatial import moving_patterns as gsmp

## Importing all other Essential Libraries
#### (DO NOT FORGET TO EXECUTE THE FUNCTIONS IN THE BOTTOM CELLS)

import psycopg2
import numpy as np
import configparser
import pandas as pd
import geopandas as gpd
import contextily as ctx
from random import choice
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.base import clone
from sklearn.cluster import DBSCAN, KMeans, MeanShift
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import silhouette_score
from shapely.geometry import Point, LineString, shape
from haversine import haversine
from datetime import datetime, timedelta

from multiprocessing import cpu_count, Pool
from functools import partial
import datetime
import pandas as pd                                                                                                                                                                                            
import numpy as np
import networkx as nx
from scipy.spatial import distance_matrix


In [174]:
import PyQt5
import matplotlib.pyplot as plt
from matplotlib import style;  style.use('ggplot')
get_ipython().magic('matplotlib qt')

In [175]:
def index_of_cluster(item, cluster_list):
    position = [ind for ind, subl in enumerate(cluster_list) if item in subl]
    if len(position)>1:
        raise ValueError
    if not position:
        return [-1]
    else:
        return position


def connected_edges(data):
    G = nx.Graph()
    G.add_edges_from(data)
    return [list(cluster) for cluster in nx.connected_components(G)]


def pairs_in_radius(df, radius):
    distances = np.triu(distance_matrix(df[['lat', 'lon']].values, df[['lat', 'lon']].values),0)
    distances[distances == 0] = np.inf
    return np.vstack(np.where(distances<=radius)).T


def get_flock_labels(timeframe,radius):
    timeframe.reset_index(drop=True , inplace=True)
    data = pairs_in_radius(timeframe, radius)
    clusters = connected_edges(data)
    timeframe['flock_label'] = timeframe.apply( lambda x: index_of_cluster(x.name, clusters)[0], axis=1)
    return timeframe


def flocks(df,radius):
    df['flock_label'] = np.nan
    df =  df.groupby('datetime', as_index=False).apply(get_flock_labels, radius)
    return df




In [176]:
def point_from_lat_lon(df_w_lat_lon):
    df_w_lat_lon['geom'] = np.nan
    df_w_lat_lon['geom'] = df_w_lat_lon[['lon', 'lat']].apply(lambda x: Point(x), axis=1)
    return gpd.GeoDataFrame(df_w_lat_lon, geometry='geom')

In [177]:
def get_correct_label(present, future):
#     cluster = present.loc[present.flock_label==label]
    lst = future.loc[future.mmsi.isin(present.mmsi)].flock_label.value_counts().index
    if lst[0] != -1 or len(lst)==1:
        return lst[0]
    else:
        return lst[1]
#         if new_label != label:
#             cmdlst.append((label, new_label))
#             tmp1 = present.flock_label
#             present['flock_label'] = present.flock_label.apply(lambda x: new_label if x==label else label if x==new_label else x)
#             tmp2 = present.flock_label
#     return cmdlst, tmp1, tmp2

In [178]:
def swap(x, pair):
    return pair[pair.index(x)-1] if x in pair else x
    

In [179]:
sample_timeFrame = pd.read_csv('4h_timeFrame_not_at_3_fing_am.csv')

In [180]:
sample_timeFrame = point_from_lat_lon(sample_timeFrame)

In [193]:
sample_timeFrame.tail()

,id,mmsi,status,turn,speed,course,heading,lon,lat,ts,geom,velocity,datetime,flock_label
20158,1.233431e+07,228186700.0,15.0,-127.0,102.3,360.0,511.0,-4.512579,48.370872,1.456832e+09,POINT (-4.5125785 48.370872),2.127987,2016-03-01 11:36:00,NaN
20159,1.233430e+07,227730220.0,15.0,-127.0,11.9,51.7,511.0,-4.596575,48.320713,1.456832e+09,POINT (-4.596575 48.320713),13.327329,2016-03-01 11:36:00,NaN
20160,1.885613e+07,227369960.0,0.0,0.0,0.1,93.5,511.0,-4.489056,48.377961,1.456832e+09,POINT (-4.489055919999998 48.3779608),0.001993,2016-03-01 11:36:00,NaN
20161,1.752261e+07,228064900.0,5.0,0.0,0.0,192.2,206.0,-4.511704,48.364637,1.456832e+09,POINT (-4.5117036 48.36463666666667),0.048667,2016-03-01 11:36:00,NaN
20162,1.752262e+07,227300000.0,7.0,0.0,0.5,313.7,293.0,-4.781028,48.100266,1.456832e+09,POINT (-4.7810283 48.100266),0.696862,2016-03-01 11:36:00,NaN


In [182]:
flocks_cluster = flocks(sample_timeFrame, 0.01)
flocks_cluster_filtered = flocks_cluster.drop(columns=['id', 'status', 'turn', 'speed', 'course', 'heading', 'lon', 'lat', 'ts', 'velocity'], axis=1) 

flocks_cluster_filtered = flocks_cluster_filtered.reset_index(drop=True)

flocks_cluster_filtered.crs = {'init': 'epsg:4326'}

tflst = []
for _, df in flocks_cluster_filtered.groupby('datetime'):
    
    tflst.append(df)

In [184]:
def fix2(df):
    grouped = df.groupby('datetime')
    for  ind, (ts, group) in enumerate(list(grouped)[:-1]):
        print (ind)
        for label, present in group.groupby('flock_label'):
            future = list(grouped)[ind+1][1]
            new_label = get_correct_label(present, future)
            df.loc[df.datetime == future.iloc[0].datetime, 'flock_label'] = future['flock_label'].apply(swap, args=((new_label, label),))
    return df, lst

In [185]:
nfd, _ = fix2(flocks_cluster_filtered)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239


In [187]:
#plots
for cnt, (ind, x) in enumerate(ndf.groupby('datetime')):
    if cnt >20: break
    try: 
        gsplt.map_plot(x.loc[x.flock_label == 0])
    except:
        continue

/home/theo/anaconda3/envs/gsa/lib/python3.6/site-packages/matplotlib/pyplot.py:522: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
